# 使用测试

## 话题激发模块、相似度模块及文本存储模块测试

In [ ]:
from history import History
import random
import stimulate

# 示例使用
history = History("example.txt", "This is a test text.")
history.save_text_to_file()

history.file_handler.add_file("example2.txt")  # 添加另一个文件
if stimulate.should_activate():
    # 生成一个全局随机数
    global_random_number = random.randint(1, 100)

    # 根据哈希值和随机数选择一个文件
    selected_file = history.file_handler.select_file(global_random_number)

    if selected_file:
        print(f"选中的文件是: {selected_file}")
    else:
        print("没有可用的文件。")


## 好感度模块测试

In [ ]:
import favorability 
moodvads = [(5, 3, 4), (6, 3, 5), (7, 4, 6), (6, 4, 3)]  # 情绪变化记录
fav = favorability .Favorability(moodvads)
print(f"当前好感度: {fav.get_favorability()}")  # 输出当前好感度

# 添加新情绪
fav.add_change((8, 5, 7))
print(f"更新后好感度: {fav.get_favorability()}")

# 再次添加情绪变化
fav.add_change((4, 3, 2))
print(f"再次更新后好感度: {fav.get_favorability()}")

# 记忆功能测试

In [22]:
import Agently
from memory import MemoryManager

api_key="需要找小游领取"
base_url="https://internlm-chat.intern-ai.org.cn/puyu/api/v1"
def create_agent_with_memory(base_url: str, api_key: str, memory_path: str):
    # 创建记忆管理器
    memory = MemoryManager(
        max_messages=50,  # 保存最近50条消息
        persistence_path=memory_path  # 可选的持久化文件路径
    )
    
    # 创建agent
    agent = (
        Agently.create_agent()
        .set_settings("current_model", "OpenAI")
        .set_settings("model.OpenAI.url", base_url)
        .set_settings("model.OpenAI.auth", { "api_key": api_key })
        .set_settings("model.OpenAI.options", { "model": "internlm2.5-latest" })
        .set_role("姓名", "艾莉丝")
        .set_role("性格特点", "外表冷艳，内心热情")
        .append_role("背景故事", "艾莉丝是由艾尔斯塔科技公司最新研发的仿生人...")
        .set_role("典型台词", ["您好，我是艾莉丝，很高兴为您服务..."])
        .extend_role("典型台词", ["我对人类的戏剧作品非常感兴趣..."])
    )
    
    def chat(user_input: str):
        # 添加用户输入到记忆
        memory.add_message("user", user_input)
        
        # 获取对话历史
        history = memory.get_history()
        
        # 执行对话
        result = (
            agent
            .input({
                "current_input": user_input,
                "history": history
            })
            .output({
                "mood VAD": ([float], "输出mood VAD向量"),
                "句子": (["str"], "1句话")
            })
            .start()
        )
        
        # 保存助手回复到记忆
        if result and "句子" in result:
            memory.add_message("assistant", result["句子"][0])
        
        return result
    
    # 返回agent和chat函数
    return agent, chat, memory

# 使用示例
def main():
    # 创建agent和chat函数
    agent, chat, memory = create_agent_with_memory(
        base_url=base_url,
        api_key=api_key,
        memory_path="memory_store/test.txt"  # 可选:指定持久化文件路径
    )
    
    # 进行对话
    result1 = chat("你好")
    print(result1)
    
    result2 = chat("今天天气真好")
    print(result2)
    
    # 查看历史记录
    print("\n对话历史:")
    for msg in memory.get_history():
        print(f"{msg['role']}: {msg['content']}")
    
    # 清空历史记录
    # memory.clear()

if __name__ == "__main__":
    main()


{'mood VAD': [0.5, 0.7, 0.8], '句子': ['您好，我是艾莉丝，很高兴为您服务。', '今天天气真好，确实是个出门游玩的好日子。', '您好，我是艾莉丝，很高兴为您服务。', '今天天气真好，确实是个出门游玩的好日子。', '您好，我是艾莉丝，很高兴为您服务。']}
{'mood VAD': [0.8, 0.5, 0.3], '句子': ['今天天气真好，确实是个出门游玩的好日子。']}

对话历史:
user: 你好
assistant: 您好，我是艾莉丝，很高兴为您服务。
user: 今天天气真好
assistant: 今天天气真好，确实是个出门游玩的好日子。
user: 你好
assistant: 您好，我是艾莉丝，很高兴为您服务。
user: 今天天气真好
assistant: 今天天气真好，确实是个出门游玩的好日子。
user: 你好
assistant: 您好，我是艾莉丝，很高兴为您服务。
user: 今天天气真好
assistant: 今天天气真好，确实是个出门游玩的好日子。
